# Simularium Conversion Tutorial : PhysiCell Data

In [1]:
from IPython.display import Image
import numpy as np
from simulariumio.physicell import PhysicellConverter, PhysicellData

This notebook provides example python code for converting your own simulation trajectories into the format consumed by the Simularium Viewer. It creates a .simularium JSON file which you can drag and drop onto the viewer like this:

![title](img/drag_drop.gif)

***
## Prepare your spatial data

The Simularium `PhysicellConverter` consumes discrete cell data from PhysiCell. 

The converter requires a `PhysicellData` object as a parameter.

`PhysicellData` contains the following:          
* **box_size** : *np.ndarray (shape = \[3\])*
    * A numpy ndarray containing the XYZ dimensions of the simulation bounding volume          
* **timestep** : *float*
    * A float amount of time in seconds that passes each step
    * Default: 0.0          
* **path_to_output_dir** : *string*
    * A string path to the PhysiCell output directory containing MultiCellDS XML and MATLAB files          
* **types** : *Dict\[int, Dict\[Any, str\]\] (optional)*
    * \[cell type ID from PhysiCell data\] : Dict\[Any, str\]
        * the cell type ID from PhysiCell data mapped to display name for that type, and display names for phases of that type
            * "name" or \[cell phase ID from PhysiCell data\] : str
                * "name" or the cell phase ID from PhysiCell data mapped to the display names
                * Default: "cell\[cell type ID from PhysiCell data\]#phase\[cell phase ID from PhysiCell data\]"          
* **scale_factor** : *float (optional)*
    * A multiplier for the ReaDDy scene, use if visualization is too large or small
    * Default: 1.0          
* **plots** : *List[Dict[str, Any]] (optional)*
    * An object containing plot data already in Simularium format

In [2]:
box_size = 1000.0

example_data = PhysicellData(
    box_size=np.array([box_size, box_size, 100.0]),
    timestep=360.0,  # seconds
    path_to_output_dir="../simulariumio/tests/data/physicell/output/",
    types={
        0 : {
            "name" : "tumor cell",
            4 : "G0G1",
        },
        1 : {
            "name" : "motile tumor cell",
            4 : "G0G1",
        }
    },
    scale_factor=0.01,
)

## Convert and save as .simularium JSON file

Once your data is shaped like in the `example_data` object, you can use the converter to generate the file at the given path:

In [3]:
PhysicellConverter(example_data).write_JSON("physicell_example")

Reading ../simulariumio/tests/data/physicell/output/output00000000.xml
Reading ../simulariumio/tests/data/physicell/output/output00000000_cells_physicell.mat
Reading ../simulariumio/tests/data/physicell/output/output00000001.xml
Reading ../simulariumio/tests/data/physicell/output/output00000001_cells_physicell.mat
Reading ../simulariumio/tests/data/physicell/output/output00000002.xml
Reading ../simulariumio/tests/data/physicell/output/output00000002_cells_physicell.mat


## Visualize in the Simularium viewer

In a supported web-browser (Firefox or Chrome), navigate to https://staging.simularium.allencell.org/ and import your file into the view.